In [ ]:
import pandas as pd 
import numpy as np
import cPickle as pk 
from collections import Counter, OrderedDict
from shapely.geometry import MultiPoint
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse, spatial
import math

In [ ]:
class OrderedCounter(Counter, OrderedDict):
    pass

In [ ]:
def learn_vocab(txS):
	#flat = ['null' if len(sublist)<1 else item for sublist in txS.values.tolist() for item in sublist]
	flat = []
	for sublist in txS.values.tolist():
		if(isinstance(sublist, float)):
			flat.append('null')
			continue
		for item in sublist:
			flat.append(item)

	dat = map(frozenset,flat)
	vocab = OrderedCounter(dat)
	#tuped = [(k,v) for k,v in vocab.items()]
	#return OrderedDict(tuped)
	return vocab
	#print vocab

In [ ]:
def gen_vec(gram_txt, vocab):
	vec = np.zeros(len(vocab)+1)
	gram_txt = map(frozenset,gram_txt)
	#print gram_txt
	if(isinstance(gram_txt,float)):
		vec[-1] = 1
		return vec
	for gram in gram_txt:
		# print gram,
		# print vocab.keys().index(gram)
		try:
			vec[vocab.keys().index(gram)]=1
		except:
			continue
	#for k,v in vocab.items():
	return vec

In [ ]:
def getcossim(namedata1, addrdata1, namedata2, addrdata2, alpha, beta):
    result = []
    for idx1,vec1 in enumerate(namedata1):
        corr={}
        for idx2,vec2 in enumerate(namedata2):
            sim_name = 1-spatial.distance.cosine(vec1,vec2)
            if math.isnan(sim_name):
                sim_name = 0.0
            sim_addr = 1-spatial.distance.cosine(addrdata1.iloc[idx1],addrdata2.iloc[idx2])
            if math.isnan(sim_addr):
                sim_addr = 0.0
            corr[idx2] = alpha*sim_name + beta*sim_addr
        most_sim = max(corr, key=corr.get)
        result.append({most_sim: corr[most_sim]})
        #print corr
    return result

In [ ]:
#Load datafiles
taap = pk.load(open("taap_params.pk","rb"))
hotbeds = pk.load(open("hotelbeds_params.pk","rb"))

In [ ]:
#Learn Vocabulary
name_voc = learn_vocab(taap['name_bigrams'])
address_voc = learn_vocab(taap['address_trigrams'])

In [ ]:
#Extract only london dataset
taap_sub = taap[taap['city']=='london']
hotbeds_sub = hotbeds[hotbeds['city']=='london']

In [ ]:
#Get vectors for hotelbeds data
hotb_namevec = hotbeds_sub['name_bigrams'].apply(gen_vec,args=(name_voc,))
hotb_addrvec = hotbeds_sub['address_trigrams'].apply(gen_vec,args=(address_voc,))
print "hotelbeds done!!"

In [ ]:
#Get vectors for TAAP data
taapb_namevec = taap_sub['name_bigrams'].apply(gen_vec,args=(name_voc,))
taapb_addrvec = taap_sub['address_trigrams'].apply(gen_vec,args=(address_voc,))
print "TAAP Done!"

In [ ]:
print taapb_addrvec.iloc[7]

In [ ]:
cossim_name = getcossim(taapb_namevec, taapb_addrvec,hotb_namevec,hotb_addrvec,0.6,1.2)
print cossim_name

In [ ]:
pk.dump(cossim_name,open("london_cossimil.pk","wb"))

In [145]:
rows= [item.keys()[0] for item in cossim_name]
row_vals = [item.values()[0] for item in cossim_name]
zero_vals = [idx for idx,x in enumerate(row_vals) if x==0.0]
print len(zero_vals)

881


In [151]:
y0= taap_sub['mapped_to_ext_id']
falsey0 = taap_sub['mapped_to_ext_id'].iloc[zero_vals]

In [152]:
y = hotbeds_sub['mapped_to_ext_id'].iloc[rows]

In [155]:
true_ctr=0.0
ctr = len(y0)
false_vals = []
for val1,val2 in zip(y0,y):
    if(val1==val2):
        true_ctr+=1
    elif val1 in falsey0:
        print val1
        ctr-=1
    else:
        #print val1,val2
        if(pd.isnull(val1)):
            ctr-=1
            continue
        false_vals.append(val1) 
print true_ctr
print len(y0)-ctr

447.0
696


In [156]:
nullvals = y0.apply(pd.isnull).tolist()
false_taap = taap_sub['mapped_to_ext_id'].isin(false_vals)

In [168]:
merged_wrongset = taap_sub[false_taap].merge(hotbeds_sub,on='mapped_to_ext_id')
merged_wrongset[['address_trigrams_x','name_bigrams_x','name_bigrams_y','address_trigrams_y']]

,address_trigrams_x,name_bigrams_x,name_bigrams_y,address_trigrams_y
0,"[(117, 129, cromwell), (129, cromwell, road)]","[(park, international), (international, hotel)]","[(park, international), (international, suites)]","[(cromwell, road, 117), (road, 117, 129)]"
1,"[(117, 129, cromwell), (129, cromwell, road)]","[(park, international), (international, hotel)]","[(park, international)]","[(cromwell, road, 117129)]"
2,"[(97, great, russell), (great, russell, street)]","[(radisson, blu), (blu, edwardian), (edwardian...","[(radisson, blu), (blu, edwardian), (edwardian...","[(great, russell, st), (russell, st, 97)]"
3,"[(17, sloane, street), (sloane, street, knight...","[(millennium, hotel), (hotel, london), (london...","[(millennium, knightsbridge)]","[(sloan, street, 17)]"
4,[],"[(mandeville, hotel)]",[],"[(mandeville, place, 814)]"
5,"[(stratton, street, mayfair)]","[(may, fair)]",[],[]
6,"[(33, beaufort, gardens), (beaufort, gardens, ...","[(beaufort, hotel)]",[],"[(beaufort, gardens, knightbridge), (gardens, ..."
7,"[(wrights, lane, kensington)]","[(holiday, inn), (inn, london), (london, kensi...","[(holiday, inn), (inn, londonkensington), (lon...",[]
8,[],[],[],[]
9,"[(101, buckingham, palace), (buckingham, palac...","[(grosvenor, hotel)]",[],"[(buckingham, palace, road), (palace, road, 101)]"
